In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi, HfFolder
import json

# Define the system message
system_message = (
    "You are a helpful, respectful, and honest assistant. Always strive to provide the most useful and safe answers possible. "
    "Ensure your responses are free from harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. "
    "Maintain a socially unbiased and positive tone. If a question is unclear or factually incoherent, explain why instead of providing an incorrect answer. "
    "If you don't know the answer to a question, avoid sharing false information."
)

# Load the dataset
dataset = load_dataset('tatsu-lab/alpaca')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42)

# Define a function to transform the data
def transform_conversation(example):
    instruction = example.get('instruction', "")
    input_text = example.get('input', "")
    output = example.get('output', "")

    llama2_text = (
        f"<s> [INST] <<SYS>> {system_message} <</SYS>>\n\n"
        f"{instruction}\n\n"
        f"{input_text} [/INST]\n"
        f"{output} </s>"
    )

    return {'text': llama2_text}

# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

# Prepare a list to store transformed examples
transformed_examples = [{'text': entry['text']} for entry in transformed_dataset]

# Save the transformed texts to a JSON file
output_file = 'transformed_llama2_prompts.json'
with open(output_file, 'w') as file:
    json.dump(transformed_examples, file, indent=4)

# Print the number of lines in the transformed dataset
print(f"Transformed dataset saved to {output_file}")
print(f"Number of lines in the transformed dataset: {len(transformed_examples)}")

# Convert the transformed examples to a Hugging Face Dataset
new_dataset = Dataset.from_dict({"text": [example["text"] for example in transformed_examples]})

# Push to Hugging Face Hub
# Note: Replace 'your-username' and 'your-dataset-name' with your actual username and desired dataset name
dataset_name = 'KushagraisTaken/alpaca_llama2'

new_dataset.push_to_hub(dataset_name)

print(f"Dataset pushed to Hugging Face Hub: {dataset_name}")


Transformed dataset saved to transformed_llama2_prompts.json
Number of lines in the transformed dataset: 52002


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/53 [00:00<?, ?ba/s]

Dataset pushed to Hugging Face Hub: KushagraisTaken/alpaca_llama2
